In [30]:
import pandas as pd
import numpy as np

data_path = "./data/House_Price.csv"
df = pd.read_csv(data_path, header=0)
# removing columns with NaN values (waterbody and n_hos_beds)
df.dropna(axis=1, inplace=True)
# removing bus_ter column (it has the same value for each row)
df.drop(["bus_ter"], axis=1, inplace=True)
# encoding airport values
airport_mapping = {'YES': 1, 'NO': 0}
df['airport'] = df['airport'].map(airport_mapping)
df.head()

,price,crime_rate,resid_area,air_qual,room_num,age,dist1,dist2,dist3,dist4,teachers,poor_prop,airport,n_hot_rooms,rainfall,parks
0,24.0,0.00632,32.31,0.538,6.575,65.2,4.35,3.81,4.18,4.01,24.7,4.98,1,11.1920,23,0.049347
1,21.6,0.02731,37.07,0.469,6.421,78.9,4.99,4.70,5.12,5.06,22.2,9.14,0,12.1728,42,0.046146
2,34.7,0.02729,37.07,0.469,7.185,61.1,5.03,4.86,5.01,4.97,22.2,4.03,0,101.1200,38,0.045764
3,33.4,0.03237,32.18,0.458,6.998,45.8,6.21,5.93,6.16,5.96,21.3,2.94,1,11.2672,45,0.047151
4,36.2,0.06905,32.18,0.458,7.147,54.2,6.16,5.86,6.37,5.86,21.3,5.33,0,11.2896,55,0.039474


In [31]:
# splitting data into train set and test set
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)

In [32]:
# regression tree implementations
import sys

# class for tree nodes
class Node:
    def __init__(self, atribute=None, threshold=None, value=None, left=None, right=None):
        self.atribute = atribute # which atribute are we splitting on
        self.threshold = threshold  # threshold value for the split
        self.value = value  # value to predict at this node (for leaf nodes)
        self.left = left  # left subtree
        self.right = right  # right subtree

# computes RSS for column of y values and y prediction (mean)
def compute_RSS(y_col: pd.DataFrame, y_pred: float):
    rss = 0
    for y in y_col:
        rss += (y-y_pred)**2
    return rss

# finds best atribute and threshold for a split
def find_best_atr_and_threshold(data: np.array, atributes: list):
    # this assumes the order of the atributes is the same as the order of data columns
    # this also assumes the target is the last atribute
    min_RSS = sys.float_info.max
    best_atribute = None
    best_threshold = -1.0
    
    for atr_index, atribute in enumerate(atributes):
        min_RSS_for_atribute = sys.float_info.max
        best_threshold_for_atribute = -1.0
        # Sorting the data by the curent column
        sorted_indices = np.argsort(data[:, atr_index])
        data = data[sorted_indices]

        for i in range(data.shape[0]-1): # TODO if column has only 0s and 1s this is a waste of time
            # splitting the data into two and calculating RSS for each
            data1 = data[:i+1, :]
            data2 = data[i+1:, :]
            cur_RSS = compute_RSS(data1[:, -1], data1[:, -1].mean()) + compute_RSS(data2[:, -1], data2[:, -1].mean())
            if cur_RSS < min_RSS_for_atribute:
                min_RSS_for_atribute = cur_RSS
                best_threshold_for_atribute = data[i, atr_index]
        if min_RSS_for_atribute < min_RSS:
            min_RSS = min_RSS_for_atribute
            best_threshold = best_threshold_for_atribute
            best_atribute = atribute
    return best_atribute, best_threshold

# builds the regression tree via recursion (it's called from fit_regression_tree function)
def fit_regression_tree_recursion(data: np.array, atributes:list, target_name:str, min_instances, max_depth, cur_depth=0): 
    if cur_depth > max_depth:
        # create leaf node and return it
        avg_y = np.mean(data[:, -1]) # calculate the mean value of all y 
        return Node(None, None, value=avg_y, left=None, right=None)
    if data.shape[0] <= min_instances:
        # create leaf node and return it
        avg_y = np.mean(data[:, -1]) # calculate the mean value of all y 
        return Node(None, None, value=avg_y, left=None, right=None)
    # get the best current atribute and best threshold
    best_atr, best_threshold = find_best_atr_and_threshold(data, atributes)
    # sort the data for best atribute
    sorted_indices = np.argsort(data[:, atributes.index(best_atr)])
    data = data[sorted_indices]
    # split the data on the threshold
    i = np.where(data[:, atributes.index(best_atr)] == best_threshold)[0][-1]
    data_left = data[:i+1, :]
    data_right = data[i+1:, :]
    # recursively build left and right subtree and append them to current node
    node_left = fit_regression_tree_recursion(data_left, atributes, target_name, min_instances, max_depth, cur_depth = cur_depth + 1)
    node_right = fit_regression_tree_recursion(data_right, atributes, target_name, min_instances, max_depth, cur_depth = cur_depth + 1)
    return Node(atribute = best_atr, threshold=best_threshold, value=None, left=node_left, right=node_right)

# prints the tree (used for testing)
def print_tree(node, depth=0, prefix="Root: "):
    if node is not None:
        print("  " * depth + prefix + f"{node.atribute} {node.threshold} {node.value}")
        if node.left or node.right:
            print_tree(node.left, depth + 1, "L-> ")
            print_tree(node.right, depth + 1, "R-> ")

# fit the  regression tree and returns the root node of the regression tree
def fit_regression_tree(data: pd.DataFrame, target_name: str, min_instances = 39, max_depth=10):
    y = data[target_name]
    X = data.drop(target_name, axis=1)
    data = pd.concat([X, y], axis=1) # putting y in the last column of data
    data = np.array(data) # converting data to np array
    atributes = X.columns.tolist() # getting the list of atributes
    return fit_regression_tree_recursion(data, atributes, target_name, min_instances, max_depth) # returns the root node of the regression tree

# gets the predicted value from a tree via recursion (called from predict_many)
def predict_recursion(node: Node, x_row: np.array, atributes: list):
    if node.value is not None: # we have reached the leaf, returning the value stored in leaf
        return node.value
    x_val = x_row[atributes.index(node.atribute)]
    if x_val <= node.threshold: # value is smaller or equal to threshold, we go to the left subtree
        return predict_recursion(node.left, x_row, atributes)
    else: # value is higher than threshold, we go to the left subtree
        return predict_recursion(node.right, x_row, atributes)

# gets the X df (without  the target) and  computes y predictions (vector)
def predict_many(root: Node, X: pd.DataFrame):
    atributes = X.columns.tolist()
    X = np.array(X)
    y_pred = np.zeros(X.shape[0])
    for i, x_row in enumerate(X):
        y_pred[i] = predict_recursion(root, x_row, atributes)
    return y_pred

In [41]:
# fitting the regression tree with train set and calculating the r2 score by predicting the test set
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

root = fit_regression_tree(df_train, "price", min_instances=39)

X_test = df_test.drop("price", axis=1)
y_test = df_test["price"]
y_pred = predict_many(root, X_test)
print("R2 score:",r2_score(y_test, y_pred))

R2 score: 0.6632328566818244


In [42]:
# test the regression tree using cross validation
# via cross validation do hyperparameter testing for min_instances and max_depth
# TODO

In [ ]:
# Compare cross validation result with the result you get while building a regression tree with scikit-learn (Use same cross validation splits on both models)
# TODO